# Tracking TCs using the UZ algorithm in TempestExtremes

**Before you start:**

1. First, make sure this notebook is opened with the bash kernel!!

2. Also, make sure you have installed TempestExtremes with instructions from `TE_intro.ipynb`.

3. Check the units of your data: below we assume that psl is in Pa, zg is in m, pressure levels are in hPa.  If it is not the case, adapt the code accordingly. You may also need to adjust variable names.

In [ ]:
# Activate the conda environment in which tempestextremes has been install
# (If error, make sure you have installed TempestExtremes as explained in `TE_intro.ipynb`).
conda activate tempestextremes

In [ ]:
# Create nodes directory if it does not exists already
if ! [ -d nodes ]
then
    mkdir nodes
fi
# Create tracks directory if it does not exists already
if ! [ -d tracks ]
then
    mkdir tracks
fi

## DetectNodes on one time step

In [ ]:
# Directory with the pre-processed simulation files for a given time step
simu_name=um_glm_n2560_RAL3p3
input_dir=/work/scratch-nopw2/sbourdin/$simu_name/data_pp
time_step=2020-01-20T00
flist=`ls $input_dir/$simu_name/*$time_step*`
echo $simu_name "--" $time_step ':' $flist

In [ ]:
# Prepare input list:
# DetectNodes needs input files to be listed with ;
INPUT=""
for f in $flist
do
INPUT="$INPUT$f;"
done
INPUT=${INPUT:0:-1} # Remove last ;
echo $INPUT

In [ ]:
echo `date`
DetectNodes \
--in_data $INPUT \
--out nodes/$time_step.dat \
--searchbymin "psl" \
--closedcontourcmd "psl,200.0,5.5,0;_DIFF(zg250,zg500),-6,6.5,1.0" \
--mergedist 6.0 \
--outputcmd "psl,min,0;_VECMAG(uas,vas),max,2"
echo `date`

## DetectNodes on several time steps

In [ ]:
# Directory with the pre-processed simulation files for a given time step
input_dir=/work/scratch-nopw2/sbourdin/data_pp/
simu_name=um_glm_n2560_RAL3p3
time_steps=`ls $input_dir/$simu_name/*psl* | cut -d '_' -f10 | cut -d '.' -f1`
for time_step in $time_steps
do
    if ! [ -f nodes/$time_step.dat ] # If file does not already exists
    then
        # Prepare file list
        flist=`ls $input_dir/$simu_name/*$time_step*`
        echo $simu_name "--" $time_step ':' $flist
        
        # Prepare input list
        INPUT=""
        for f in $flist
        do
        INPUT="$INPUT$f;"
        done
        INPUT=${INPUT:0:-1} # Remove last ;
        echo $INPUT
    
        # DetectNodes
        DetectNodes \
            --in_data $INPUT \
            --out nodes/$time_step.dat \
            --searchbymin "psl" \
            --closedcontourcmd "psl,200.0,5.5,0;_DIFF(zg250,zg500),-6,6.5,1.0" \
            --mergedist 6.0 \
            --outputcmd "psl,min,0;_VECMAG(uas,vas),max,2"
    else
        echo "$time_step already computed"
    fi
done

## StitchNodes

In [ ]:
ls nodes/*

In [ ]:
flist=`ls nodes/*`
cat $flist > all_nodes.dat
StitchNodes \
--in all_nodes.dat \
--out tracks/$simu_name.csv \
--out_file_format "csv" \
--in_fmt "lon,lat,slp,wind" \
--range 8.0 \
--mintime "54h" \
--maxgap "24h" \
--threshold "wind,>=,10.0,10;lat,<=,50.0,10;lat,>=,-50.0,10"